In [ ]:
#Final OpenSource Code

import pandas as pd
import requests
import json

def get_chromium_data(user):
    chromium_url = f"https://chromium-review.googlesource.com/changes/?q=author:{user}"
    print("Invoking chromium-review web API...", chromium_url)
    response_chromium = requests.get(chromium_url)
    if response_chromium.status_code == 200:
        output_chromium = response_chromium.text
        output_json_chromium = json.loads(output_chromium[4:])
        if isinstance(output_json_chromium, list) and len(output_json_chromium) > 0:
            chromium_data = []
            for item in output_json_chromium:
                if isinstance(item, dict):
                    chromium_data.append({
                        "subject": item.get("subject", ""),
                        "project": item.get("project", ""),
                        "branch": item.get("branch", ""),
                        "status": item.get("status", ""),
                        "updated": item.get("updated", ""),
                        "User": user,
                        "Owner": user.split("@")[0],
                        "Repo": "Chromium",
                        "URL": chromium_url
                    })
                else:
                    print("Unexpected format of JSON data from chromium-review.")
        else:
            print("No data returned or unexpected format of JSON data from chromium-review.")
            # Update chromium_url to search by author instead of owner
            chromium_url = f"https://chromium-review.googlesource.com/changes/?q=author:{user}"
            print("Invoking chromium-review web API...", chromium_url)
            response_chromium = requests.get(chromium_url)
            output_chromium = response_chromium.text
            output_json_chromium = json.loads(output_chromium[4:])
            if isinstance(output_json_chromium, list) and len(output_json_chromium) > 0:
                chromium_data=[]
                for item in output_json_chromium:
                    if isinstance(item, dict):
                        chromium_data.append({
                        "subject": item.get("subject", ""),
                        "project": item.get("project", ""),
                        "branch": item.get("branch", ""),
                        "status": item.get("status", ""),
                        "updated": item.get("updated", ""),
                        "User": user,
                        "Owner": user.split("@")[0],
                        "Repo": "Chromium",
                        "URL": chromium_url
                    })
                    else:
                        print("Unexpected format of JSON data from chromium-review.")
        return chromium_data
    else:
        print(f"Failed to fetch data from chromium-review for user: {user}")

    return []

def get_android_data(user):
    android_url = f"https://android-review.googlesource.com/changes/?q=owner:{user}"
    print("Invoking android-review web API...", android_url)
    response_android = requests.get(android_url)
    
    if response_android.status_code == 200:
        output_android = response_android.text
        output_json_android = json.loads(output_android[4:])
        
        if isinstance(output_json_android, list) and len(output_json_android) > 0:
            android_data = []
            for item in output_json_android:
                if isinstance(item, dict):
                    android_data.append({
                        "subject": item.get("subject", ""),
                        "project": item.get("project", ""),
                        "branch": item.get("branch", ""),
                        "status": item.get("status", ""),
                        "updated": item.get("updated", ""),
                        "User": user,
                        "Owner": user.split("@")[0],
                        "Repo": "Android",
                        "URL": android_url
                    })
                else:
                    print("Unexpected format of JSON data from android-review.")
            return android_data
        else:
            print("No data returned or unexpected format of JSON data from android-review.")
    else:
        print(f"Failed to fetch data from android-review for user: {user}")
    return []

def get_gitlab_data(user):
    gitlab_url = f"https://gitlab.freedesktop.org/api/v4/projects/176/repository/commits?author={user}"
    print("Invoking gitlab web API...", gitlab_url)
    response_gitlab = requests.get(gitlab_url)
    
    if response_gitlab.status_code == 200:
        output_gitlab = response_gitlab.text
        output_json_gitlab = json.loads(output_gitlab)
        
        if isinstance(output_json_gitlab, list) and len(output_json_gitlab) > 0:
            gitlab_data = []
            for item in output_json_gitlab:
                if isinstance(item, dict):
                    gitlab_data.append({
                        "subject": item.get("title", ""),
                        "project": "",  # You can update this based on Gitlab API response
                        "branch": "",  # You can update this based on Gitlab API response
                        "status": "MERGED",  # You can update this based on Gitlab API response
                        "updated": item.get("committed_date", ""),
                        "User": user,
                        "Owner": user.split("@")[0],
                        "Repo": "GitLab",
                        "URL": gitlab_url
                    })
                else:
                    print("Unexpected format of JSON data from gitlab.")
            return gitlab_data
        else:
            print("No data returned or unexpected format of JSON data from gitlab.")
    else:
        print(f"Failed to fetch data from gitlab for user: {user}")
    return []



def get_github_data(user_email, repo_name):
  
 

    if user_email:
        # Use "/repos/{repo_name}/commits" for all commits (replace if needed)
        github_url = f"https://api.github.com/repos/{repo_name}/commits"
        params = {"author": user_email}
        try:
            response_github = requests.get(github_url, params=params)
            response_github.raise_for_status()  # Raise an exception for non-200 status codes
            output_github = response_github.json()
            if isinstance(output_github, list):
                if len(output_github) > 0:
                    commits_data = []
                    for commit in output_github:
                        if isinstance(commit, dict):
                            commits_data.append({
                                "subject": commit.get("commit", {}).get("message", ""),
                                "project": "",  # You can update this based on GitHub API response
                                "branch": "",  # You can update this based on GitHub API response
                                "status": "MERGED",  # You can update this based on GitHub API response
                                "updated": commit.get("commit", {}).get("committer", {}).get("date", ""),
                                "User": user_email,
                                "Owner": user_email.split("@")[0],
                                "Repo": "GitHub",
                                "URL": github_url
                            })
                        else:
                            print("Unexpected format of commit data from GitHub API.")
                    commits_df = pd.DataFrame(commits_data)
                    return commits_df
                else:
                    print(f"No commits found for the user ({user_email}) in the repository ({repo_name}).")
            else:
                print("Unexpected response format from GitHub API.")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from GitHub: {e}")
    else:
        print("No email provided.")

    return None

user_email = "something@anything.com"

repo_name= [
    "Project 1",
    "Project 2",
    "Project 3"
    
]

emails_df = pd.read_excel(r"Excel file path where you have saved names of the owner.xlsx")
users = emails_df["Email"].tolist()

chromium_data_list = [get_chromium_data(user) for user in users]
chromium_data_df = pd.concat([pd.DataFrame(data) for data in chromium_data_list], ignore_index=True)

android_data_list = [get_android_data(user) for user in users]
android_data_df = pd.concat([pd.DataFrame(data) for data in android_data_list], ignore_index=True)

gitlab_data_list = [get_gitlab_data(user) for user in users]
gitlab_data_df = pd.concat([pd.DataFrame(data) for data in gitlab_data_list], ignore_index=True)

# Assuming `get_github_data` and other variables are defined earlier in your code

github_data_list = [get_github_data(user, repo) for user in users for repo in repo_name]
github_data_df = pd.concat([pd.DataFrame(data) for data in github_data_list], ignore_index=True)



# Consolidate all data into a single Excel file
consolidated_df = pd.concat([chromium_data_df, android_data_df, gitlab_data_df, github_data_df], ignore_index=True)
consolidated_df.to_excel(r"Excel file path where you want to save the data.xlsx", index=False, engine='openpyxl')

